In [35]:
import losses
import torch
import utilities
import dataloader
import torchvision
torch.manual_seed(42)


In [43]:
loss_network = losses.LossNetwork(layers=[2,9,16]).cuda()
critereon = losses.PerceptualLoss(loss_network)
l1_loss = torch.nn.L1Loss()

In [9]:
G = utilities.get_sepconv(input_size = 2)
G = G.cuda().eval()

In [12]:
ds = dataloader.adobe240_dataset( quadratic = False )

N_train = int(len(ds) * 0.8)
N_valid = int(len(ds) * 0.1)
N_test = len(ds) - N_train - N_valid

train, valid, test = torch.utils.data.random_split(ds, [N_train, N_valid, N_test])

train = dataloader.TransformedDataset(train, crop_size=(128,128), normalize=True)
train_dl = torch.utils.data.DataLoader(train, batch_size=2, pin_memory=True, shuffle=True)

In [14]:
for X,y in train_dl:
    X = X.cuda()
    y = y.cuda()
    break

In [39]:
y_hat = G(X)

tc torch.Size([2, 64, 64, 64])


In [44]:
loss = critereon(y_hat, y)
l1   = l1_loss(y_hat,y)

In [45]:
loss, l1

(tensor(0.0076, device='cuda:0', grad_fn=<AddBackward0>),
 tensor(0.0148, device='cuda:0', grad_fn=<L1LossBackward>))

In [37]:
vgg = torchvision.models.vgg16_bn(pretrained=True)
vgg

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256